In [1]:
import contextlib
from pprint import pprint as pp
import sys
import dateutil.parser as dt_parser

import pandas as pd
from pymongo import MongoClient

In [2]:
@contextlib.contextmanager
def mongo_client():
    # <ENTER>
    client = MongoClient('')
#     print('clr数据库连接成功！')
    try:
        # Like __enter__()'s return statement
        yield client
        # <NORMAL EXIT>
        client.close()
#         print('clr关闭数据库！')
    except Exception:
        # <EXCEPTIONAL EXIT>
        print('mongo_client: exceptional exit', sys.exc_info())
        raise

In [3]:
SEARCH_SUMMARY_COLL = 'stats_search_summary'
STAT_TYPES = ('search_count', 'search_click', 'search_transform', 'recomm_click', 'recomm_transform', 'index_count')

In [4]:
time_from = dt_parser.parse('2022-10-31')
time_to = dt_parser.parse('2022-11-30')

In [5]:
with mongo_client() as mongo:
    d = mongo.get_default_database()
    docs = d[SEARCH_SUMMARY_COLL].find({'$and': [{'time': {'$gte': time_from}}, {'time': {'$lte': time_to}}]})
    records = []
    for doc in docs:
        try:
            _type = doc['type']
            _count = doc['count']
            _time = doc['time']
        except KeyError:
            print(doc)
            break
        records.append({'type': _type, 'count': _count, 'time': _time})
    df = pd.DataFrame(records)

In [6]:
# datetime_df = df.copy(deep=True)

In [7]:
df.head(n=10)

,type,count,time
0,search_count,385,2022-10-31
1,search_click,91,2022-10-31
2,search_transform,63,2022-10-31
3,recomm_click,65,2022-10-31
4,recomm_transform,9,2022-10-31
5,index_count,1,2022-10-31
6,search_count,16,2022-11-01
7,search_click,0,2022-11-01
8,search_transform,0,2022-11-01
9,recomm_click,0,2022-11-01


In [8]:
df.index

RangeIndex(start=0, stop=186, step=1)

In [9]:
df = df.set_index('time').sort_index()

In [10]:
df.index

DatetimeIndex(['2022-10-31', '2022-10-31', '2022-10-31', '2022-10-31',
               '2022-10-31', '2022-10-31', '2022-11-01', '2022-11-01',
               '2022-11-01', '2022-11-01',
               ...
               '2022-11-29', '2022-11-29', '2022-11-29', '2022-11-29',
               '2022-11-30', '2022-11-30', '2022-11-30', '2022-11-30',
               '2022-11-30', '2022-11-30'],
              dtype='datetime64[ns]', name='time', length=186, freq=None)

In [11]:
df.head(n=10)

,type,count
time,,
2022-10-31,search_count,385
2022-10-31,search_click,91
2022-10-31,search_transform,63
2022-10-31,recomm_click,65
2022-10-31,recomm_transform,9
2022-10-31,index_count,1
2022-11-01,search_count,16
2022-11-01,search_click,0
2022-11-01,search_transform,0


In [12]:
pd.options.display.max_rows = None

In [13]:
df = df.groupby(by=['type']).resample('1D').agg({'count': 'sum'})
df

count
type             time             
index_count      2022-10-31      1
                 2022-11-01      1
                 2022-11-02      1
                 2022-11-03      1
                 2022-11-04      1
                 2022-11-05      1
                 2022-11-06      1
                 2022-11-07      1
                 2022-11-08      1
                 2022-11-09      1
                 2022-11-10      1
                 2022-11-11      1
                 2022-11-12      1
                 2022-11-13      1
                 2022-11-14      1
                 2022-11-15      1
                 2022-11-16      1
                 2022-11-17      1
                 2022-11-18      1
                 2022-11-19      1
                 2022-11-20      1
                 2022-11-21      1
                 2022-11-22      1
                 2022-11-23      1
                 2022-11-24      1
                 2022-11-25      1
                 2022-11-26      1
                 2022-11-27      1
                 2022-11-28      1
                 2022-11-29      1
                 2022-11-30      1
recomm_click     2022-10-31     65
                 2022-11-01      0
                 2022-11-02      0
                 2022-11-03      0
                 2022-11-04      0
                 2022-11-05      0
                 2022-11-06      0
                 2022-11-07      0
                 2022-11-08      0
                 2022-11-09      0
                 2022-11-10      0
                 2022-11-11      0
                 2022-11-12      0
                 2022-11-13      0
                 2022-11-14      0
                 2022-11-15      0
                 2022-11-16      0
                 2022-11-17      0
                 2022-11-18      0
                 2022-11-19      0
                 2022-11-20      0
                 2022-11-21      0
                 2022-11-22      0
                 2022-11-23      0
                 2022-11-24      0
                 2022-11-25      0
                 2022-11-26      0
                 2022-11-27      0
                 2022-11-28      0
                 2022-11-29      0
                 2022-11-30      0
recomm_transform 2022-10-31      9
                 2022-11-01      0
                 2022-11-02      0
                 2022-11-03      0
                 2022-11-04      0
                 2022-11-05      0
                 2022-11-06      0
                 2022-11-07      0
                 2022-11-08      0
                 2022-11-09      0
                 2022-11-10      0
                 2022-11-11      0
                 2022-11-12      0
                 2022-11-13      0
                 2022-11-14      0
                 2022-11-15      0
                 2022-11-16      0
                 2022-11-17      0
                 2022-11-18      0
                 2022-11-19      0
                 2022-11-20      0
                 2022-11-21      0
                 2022-11-22      0
                 2022-11-23      0
                 2022-11-24      0
                 2022-11-25      0
                 2022-11-26      0
                 2022-11-27      0
                 2022-11-28      0
                 2022-11-29      0
                 2022-11-30      0
search_click     2022-10-31     91
                 2022-11-01      0
                 2022-11-02      0
                 2022-11-03      0
                 2022-11-04      0
                 2022-11-05      0
                 2022-11-06      0
                 2022-11-07      0
                 2022-11-08      0
                 2022-11-09      2
                 2022-11-10      5
                 2022-11-11      0
                 2022-11-12      0
                 2022-11-13      0
                 2022-11-14      0
                 2022-11-15      0
                 2022-11-16      0
                 2022-11-17      0
                 2022-11-18      0
                 2022-11-19      0
    

In [14]:
delta = pd.to_datetime(time_to) - pd.to_datetime(time_from) + pd.to_timedelta('1D')
length = delta.days
default = pd.DataFrame([{'count': 0}] * length, index=pd.date_range(start=time_from, end=time_to))
# default

In [15]:
dfs = []
keys = set()
for index, row in df.iterrows():
    if index[0] in keys:
        continue
    _df = (
        (df.loc[index[0]] + default).fillna(0).astype('int32')
            .assign(type=pd.Series([index[0]] * length, index=default.index))
    )
#     print(_df)
    dfs.append(_df)
    keys.add(index[0])
#     break

In [16]:
df = pd.concat(dfs)
df = df.reset_index()
df

,time,count,type
0,2022-10-31,1,index_count
1,2022-11-01,1,index_count
2,2022-11-02,1,index_count
3,2022-11-03,1,index_count
4,2022-11-04,1,index_count
5,2022-11-05,1,index_count
6,2022-11-06,1,index_count
7,2022-11-07,1,index_count
8,2022-11-08,1,index_count
9,2022-11-09,1,index_count


In [17]:
df['time'] = df['time'].dt.date

In [18]:
type(df['time'][0])

datetime.date

In [19]:
df = df.set_index(['type', 'time']).sort_index(level=['type', 'time'])
df

count
type             time             
index_count      2022-10-31      1
                 2022-11-01      1
                 2022-11-02      1
                 2022-11-03      1
                 2022-11-04      1
                 2022-11-05      1
                 2022-11-06      1
                 2022-11-07      1
                 2022-11-08      1
                 2022-11-09      1
                 2022-11-10      1
                 2022-11-11      1
                 2022-11-12      1
                 2022-11-13      1
                 2022-11-14      1
                 2022-11-15      1
                 2022-11-16      1
                 2022-11-17      1
                 2022-11-18      1
                 2022-11-19      1
                 2022-11-20      1
                 2022-11-21      1
                 2022-11-22      1
                 2022-11-23      1
                 2022-11-24      1
                 2022-11-25      1
                 2022-11-26      1
                 2022-11-27      1
                 2022-11-28      1
                 2022-11-29      1
                 2022-11-30      1
recomm_click     2022-10-31     65
                 2022-11-01      0
                 2022-11-02      0
                 2022-11-03      0
                 2022-11-04      0
                 2022-11-05      0
                 2022-11-06      0
                 2022-11-07      0
                 2022-11-08      0
                 2022-11-09      0
                 2022-11-10      0
                 2022-11-11      0
                 2022-11-12      0
                 2022-11-13      0
                 2022-11-14      0
                 2022-11-15      0
                 2022-11-16      0
                 2022-11-17      0
                 2022-11-18      0
                 2022-11-19      0
                 2022-11-20      0
                 2022-11-21      0
                 2022-11-22      0
                 2022-11-23      0
                 2022-11-24      0
                 2022-11-25      0
                 2022-11-26      0
                 2022-11-27      0
                 2022-11-28      0
                 2022-11-29      0
                 2022-11-30      0
recomm_transform 2022-10-31      9
                 2022-11-01      0
                 2022-11-02      0
                 2022-11-03      0
                 2022-11-04      0
                 2022-11-05      0
                 2022-11-06      0
                 2022-11-07      0
                 2022-11-08      0
                 2022-11-09      0
                 2022-11-10      0
                 2022-11-11      0
                 2022-11-12      0
                 2022-11-13      0
                 2022-11-14      0
                 2022-11-15      0
                 2022-11-16      0
                 2022-11-17      0
                 2022-11-18      0
                 2022-11-19      0
                 2022-11-20      0
                 2022-11-21      0
                 2022-11-22      0
                 2022-11-23      0
                 2022-11-24      0
                 2022-11-25      0
                 2022-11-26      0
                 2022-11-27      0
                 2022-11-28      0
                 2022-11-29      0
                 2022-11-30      0
search_click     2022-10-31     91
                 2022-11-01      0
                 2022-11-02      0
                 2022-11-03      0
                 2022-11-04      0
                 2022-11-05      0
                 2022-11-06      0
                 2022-11-07      0
                 2022-11-08      0
                 2022-11-09      2
                 2022-11-10      5
                 2022-11-11      0
                 2022-11-12      0
                 2022-11-13      0
                 2022-11-14      0
                 2022-11-15      0
                 2022-11-16      0
                 2022-11-17      0
                 2022-11-18      0
                 2022-11-19      0
    

In [20]:
df.to_records()

rec.array([('index_count', datetime.date(2022, 10, 31),   1),
           ('index_count', datetime.date(2022, 11, 1),   1),
           ('index_count', datetime.date(2022, 11, 2),   1),
           ('index_count', datetime.date(2022, 11, 3),   1),
           ('index_count', datetime.date(2022, 11, 4),   1),
           ('index_count', datetime.date(2022, 11, 5),   1),
           ('index_count', datetime.date(2022, 11, 6),   1),
           ('index_count', datetime.date(2022, 11, 7),   1),
           ('index_count', datetime.date(2022, 11, 8),   1),
           ('index_count', datetime.date(2022, 11, 9),   1),
           ('index_count', datetime.date(2022, 11, 10),   1),
           ('index_count', datetime.date(2022, 11, 11),   1),
           ('index_count', datetime.date(2022, 11, 12),   1),
           ('index_count', datetime.date(2022, 11, 13),   1),
           ('index_count', datetime.date(2022, 11, 14),   1),
           ('index_count', datetime.date(2022, 11, 15),   1),
           ('inde